# Induktive Statistik & Gruppenvergleiche

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from pathlib import Path

In [13]:
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 1. DATEN LADEN & VORBEREITUNG

Wir nutzen den Snapshot-Datensatz, da Hypothesentests unabhängige Beobachtungen erfordern. Zeitreihendaten würden durch die Wiederholung der Länder (Autokorrelation) die p-Werte verfälschen (Pseudoreplikation).

In [14]:
current_dir = Path.cwd()
data_path = current_dir.parent / 'data' / 'oecd_snapshot_latest.csv'

try:
    df = pd.read_csv(data_path)
    print("Daten erfolgreich geladen.")

    df = df.loc[:, ~df.columns.duplicated()]
        
except FileNotFoundError:
    print(f"FEHLER: Datei nicht gefunden unter {data_path}")

Daten erfolgreich geladen.


In [15]:
target_var = "Life satisfaction"  # Unsere abhängige Hauptvariable (Y)
group_var_metric = "GDP per capita" # Variable, die wir nutzen, um Gruppen zu bilden (X)

print(f"Analyse-Fokus: Zielvariable = '{target_var}' | Gruppen-Indikator = '{group_var_metric}'")

Analyse-Fokus: Zielvariable = 'Life satisfaction' | Gruppen-Indikator = 'GDP per capita'


## 2. INDUKTIVE STATISTIK: PUNKSCHÄTZER & KONFIDENZINTERVALLE

Wir wollen nicht nur den Mittelwert unserer Stichprobe kennen, sondern schätzen, wo der wahre Mittelwert der Population liegt.

In [17]:
def calculate_confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return m, m - h, m + h

mean_val, lower, upper = calculate_confidence_interval(df[target_var].dropna())

print(f"\n--- Konfidenzintervall (95%) für {target_var} ---")
print(f"Punktschätzer (Mittelwert): {mean_val:.2f}")
print(f"95% Konfidenzintervall: [{lower:.2f}, {upper:.2f}]")
print("Interpretation: Wir sind zu 95% sicher, dass der wahre Mittelwert aller OECD-ähnlichen Länder in diesem Intervall liegt.")

KeyError: 'Life satisfaction'